# Crear Tablas 

Los objetivos de aprendizaje son:

1. Crear modelo de datos
2. Definir campos y columnas
3. Crear el DB Engine
4. Crear la Base de Datos y la Tabla

## Crear modelo de datos

Lo primero que debemos hacer es crear una clase para representar los datos en la tabla, a esto se le conoce como modelo de datos.

Tomaremos como ejemplo la tabla `asegurados` que vimos antes.

In [1]:
from typing import Optional
from sqlmodel import SQLModel, Field

class Asegurado(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    nombre: str
    lob: str
    edad: Optional[int] = None

- La clase `Asegurado` representa la tabla asegurados. Cada instancia de esta clase representará un registro de la tabla.
<br>

- Usamos la configuración `table=True` para decirle a `SQLModel` que la clase `Asegurado` es una tabla.

## Definir columnas

El siguiente paso es definir las columnas de la clase usando atributos y anotaciones.

- El nombre de cada atributo será el nombre de la columna de la tabla.
<br>

- El tipo de cada atributo será también el tipo de columna de la tabla.
<br>

``` python
    id: Optional[int] = Field(default=None, primary_key=True)
    nombre: str
    lob: str
    edad: Optional[int] = None
```

- `edad: Optional[int] = None`: Significa que la columna no es obligatoria, acepta valores enteros y por defecto tendrá valor `NULL`.
<br>

- `id: Optional[int] = Field(default=None, primary_key=True)`: Es la **primary key** de la tabla, para indicarlo usamos la clase `Field` con el atributo `primary_key=True`.

>**Nota**: usamos `Optional` como anotación de `id` porque su valor no lo genera nuestro código, sino la aplicación de BD.

- El resto de columnas sólo son del tipo `TEXT` en la base de datos.


## Crear el DB Engine

Para interactuar con la base de datos usando `SQLAlchemy` necesitamos crear un `Engine`. 

>**Nota**: SQLModel está construido sobre SQLAlchemy

El engine es un objeto que gestiona:

- La conexión con la base de datos.

- La traducción de nuestros comandos al dialecto de la BD.

Crear el motor es muy simple, sólo tenemos que llamar a la función `create_engine()` con una URL de la base de datos:

In [2]:
from sqlmodel import create_engine

sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=True)
engine

Engine(sqlite:///database.db)

Cada base de datos soportada por `SQLAlchemy` tiene su propio tipo de URL. Nosotros estamos usando SQLite porque normalmente viene pre-instalada en todos lo sistemas operativos. 

Para SQLite usamos `sqlite:/// ` segida del path de la base de datos: 

- `"sqlite:///database.db"`
- `"sqlite:///databases/local/application.db"`

En este ejemplo, también estamos usando el argumento `echo=True`. Hará que el motor imprima todos los comandos SQL.


>**Nota**: Solo debería existir una única instancia de la clase `Engine` en nuestra aplicación que se reutiliza en todas partes. Hay otra clase llamada `Session` que nos ayuda a gestionar un único `Engine`, la veremos más adelante.



La [documentación de SQLAlchemy](https://docs.sqlalchemy.org/en/20/core/engines.html) contiene los detalles de configuración para conectar con otras bases de datos.


## Crear la Base de Datos y la Tabla

Crear el `engine` no creará el archivo `database.db`. Para ello deberemos de ejecutar el comando:

```python
SQLModel.metadata.create_all(engine)
```

Al ejecutarlo se creará también la tabla `asegurado`.


### Metadata

La clase `SQLModel` tiene un atributo llamado `metadata`, que es una instancia de la clase [`MetaData`](https://docs.sqlalchemy.org/en/14/core/metadata.html).

La clase `MetaData` es un contenedor que almacena la información que describe la base de datos, e.g. tablas, columnas, nombre de la base de datos, etc.

Cuando creamos una clase que hereda de `SQLModel` con la configuración `table = True`, ésta quedará registradas en el atributo `metadata` de la clase `SQLModel`.


### Método `.create_all()`

Toma un `engine` y lo usa para crear la base de datos y todas las tablas registradas dentro de la instancia de la clase `MetaData`.

>**Nota**: Siempre deberemos llamar al método `.create_all()` después de crear todas las clases que representen tablas dentro de la base de datos.

In [3]:
!python crear_base_de_datos.py

2023-02-06 18:52:40,316 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-06 18:52:40,316 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("asegurado")
2023-02-06 18:52:40,316 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-06 18:52:40,316 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("asegurado")
2023-02-06 18:52:40,317 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-06 18:52:40,318 INFO sqlalchemy.engine.Engine 
CREATE TABLE asegurado (
	id INTEGER NOT NULL, 
	nombre VARCHAR NOT NULL, 
	lob VARCHAR NOT NULL, 
	edad INTEGER, 
	PRIMARY KEY (id)
)


2023-02-06 18:52:40,318 INFO sqlalchemy.engine.Engine [no key 0.00014s] ()
2023-02-06 18:52:40,336 INFO sqlalchemy.engine.Engine COMMIT
